1.2 жадный выбор, начиная со случайного груза

2.2 выбрать только 20% самых приспособленных особей

3.2 однородный (каждый бит от случайно выбранного родителя)

4.1 инвертирование всех битов у 1 особи

5.2 «штраф» за «старость» -10% функции приспособленности, выбор лучших

In [1]:
import numpy as np
import pandas as pd
import random
from random import randint

In [2]:
df = pd.read_csv('13.txt', delimiter=' ', engine='python')
wv = df.columns.tolist()
df.columns = ['v', 'c']
df['w'] = df.index
data = df[['w','v','c']].values.tolist()
data = pd.DataFrame.from_records(data, columns=['w','v','c'])

data

,w,v,c
0,993.0,1.0,275.0
1,285.0,0.7,168.0
2,1230.0,0.8,245.0
3,1139.0,1.1,225.0
4,736.0,0.5,339.0
5,524.0,0.7,220.0
6,229.0,0.5,153.0
7,372.0,0.4,191.0
8,466.0,1.2,179.0
9,543.0,0.7,184.0


In [3]:
#Начальная популяция
initialPopulation = 200 
#Количество генов
numGenes = len(data) 
#Грузоподъемность
W = int(wv[0]) 
#Вместимость
V = int(wv[1]) 

In [4]:
#Проверка ограничения на грузоподъемность
def checkingW(person):
    sumW = 0
    for i in range (0, numGenes):
        sumW += data.at[i,'w'] * person[i]
    if sumW > W:
        return 0
    else: 
        return 1

#Проверка ограничения на вместимость
def checkingV(person):
    sumV = 0
    for i in range (0, numGenes):
        sumV += data.at[i,'v'] * person[i]
    if sumV > V:
        return 0
    else: 
        return 1

In [5]:
#Начальная популяция: жадный выбор, начиная со случайного груза

#Создание особи
def createPerson():
    person = []
    ind = randint(0, numGenes - 1)
    w, v = 0, 0
    i = 0
    while i < numGenes:
        if i < ind or w >= W or v >= V:
            person.append(0)
            i += 1
        else:
            w += data.at[i,'w']
            v += data.at[i,'v']
            if w < W and v < V:
                person.append(1)
                i += 1
    return person

#Создание начальной популяции
def createInitialPopulation():
    return [createPerson() for _ in range(0, initialPopulation)]

In [6]:
#Определение ценности особи
def personC(person):
    sumC = 0
    for i in range(0, numGenes):
        sumC += data.at[i,'c'] * person[i]
    return sumC

In [7]:
#Выбрать только 20% самых приспособленных особей
def selection(population):
    population.sort(key=lambda x: personC(x), reverse=True)
    newPopulation = []
    for i in range (0, round(len(population) * 0.2)):
            newPopulation.append(population[i])
    return newPopulation

In [8]:
#Однородное скрещивание (каждый бит от случайно выбранного родителя)

#Создание потомка
def newChild(mother, father):
    child = []
    mask = createPerson()
    for i in range(0, len(mask)):
        if mask[i] == 1:
            child.append(mother[i])
        else:
            child.append(father[i])
    return child

#Скрещивание
def crossingover(population):
    childs = []
    while len(population) > 0:
        random.shuffle(population)
        mother = population.pop()
        father = population.pop()
        child1 = newChild(mother, father)
        child2 = newChild(mother, father)
        if checkingV(child1) and checkingW(child1):
            childs.append(child1)
        if checkingV(child2) and checkingW(child2):
            childs.append(child2)
        return childs

In [9]:
#Мутация. Инвертирование всех битов у 1 особи

def mutation(person):
    for i in range(0, len(person)):
        if person[i] == 0:
            person[i] = 1
        else:
            person[i] = 0
    return person

In [10]:
def GA():

    #Оценка результата: Наступила сходимость 
    #(функция приспособленности лучшей особи в популяциях отличается не более, чем на стоимость самой дешевой вещи) 
    #или прошло 500 поколений
    c = []
    C = 0
    for i in range (0, len(data)):
        c.append(data.at[i,'c'])
    
    minC = min(c)
    oldC = 0
    newC = 99999999
    maxIteration = 500

    while maxIteration > 0 and abs(oldC - newC) > minC:
        oldC = newC

        # Начальная популяция
        population = createInitialPopulation()
        # Отбор
        population = selection(population)
        # Скрещивание
        childs = crossingover(population)
        # Мутация
        rndPersonInd = randint(0, len(population) - 1)
        population[rndPersonInd] = mutation(population[rndPersonInd])
        # Новая популяция
        for person in population:
            person.append(personC(person) * 0.9)
        for child in childs:
            child.append(personC(child))
            population.extend(childs) #Добавление потомков к популяции
            population = list(filter(lambda x: checkingW(x) and checkingV(x), population)) #Проверка на ограничения
            population.sort(key=lambda x: personC(x), reverse=True) #Сортировка по ценности
            population=population[0:200] #Популяция = 200 особей
            bestPopulation = population[0] #Лучшее решение

        newC = bestPopulation[-1]
        maxIteration -= 1

    return bestPopulation

In [11]:
bestPopulation = []
bestPopulation = GA()

In [12]:
weight, volume, value = 0, 0, 0
best = []
for i in range(numGenes):
    weight += bestPopulation[i] * data.at[i,'w']
    volume += bestPopulation[i] * data.at[i,'v']
    value += bestPopulation[i] * data.at[i,'c']
    if bestPopulation[i] == 1:
        best.append([data.at[i,'w'], data.at[i,'v'], data.at[i,'c']])
result = pd.DataFrame.from_records(best, columns=['w','v','c'])

print(result)
print("Вес:", weight)
print("Объем:", round(volume,1))
print("Ценность:", value)

         w    v      c
0    390.0  0.6  119.0
1    775.0  1.0  290.0
2    795.0  0.9  337.0
3   1058.0  0.5  336.0
4    176.0  0.7  342.0
5    927.0  0.9  249.0
6    482.0  0.6  226.0
7    714.0  0.7  196.0
8    801.0  0.6  234.0
9   1044.0  1.1  194.0
10  1550.0  0.9  378.0
11   950.0  0.9  364.0
12   447.0  0.8  263.0
13   707.0  1.0  199.0
14  1656.0  0.7  187.0
Вес: 12472.0
Объем: 11.9
Ценность: 3914.0


In [13]:
file=open('GA_4_2.txt','w', encoding='utf-8')
file.write(result.to_string())
file.write("\nВес: " + str(weight))
file.write("\nОбъем: " + str(round(volume,1)))
file.write("\nЦенность: " + str(value))
file.close()